In [22]:
%%capture
%%pip install pulp

In [121]:
from pulp import *
import pulp
import pandas as pd
import numpy as np
import random
import math

In [25]:


# Lemos o ficheiro com os resultados das previsões dos produtos
df_productPredictions = pd.read_excel('./ml/results/results.xlsx')

# Verificamos o dataframe obtido
print(df_productPredictions.head())



  product_id                  model    rmse_sales  rmse_revenue     mae_sales  \
0      P0001       LinearRegression  4.885453e-01  2.578147e+00  4.041303e-01   
1      P0001  RandomForestRegressor  4.541629e+00  2.643378e+01  2.912128e+00   
2      P0001           XGBRegressor  3.921948e+00  2.690934e+01  2.526265e+00   
3      P0002       LinearRegression  1.218277e-15  2.703535e-14  1.016386e-15   
4      P0002  RandomForestRegressor  1.381411e+00  2.095142e+02  3.619048e-01   

    mae_revenue  r2_sales  r2_revenue  periodicity_total_pred_sales  \
0  2.147004e+00  0.999275    0.999468                            20   
1  1.623660e+01  0.937366    0.944062                            32   
2  1.796265e+01  0.953292    0.942031                            32   
3  1.974587e-14  1.000000    1.000000                             1   
4  6.351167e+01  0.380296    0.511473                             0   

   periodicity_total_pred_revenue  
0                          196.86  
1             

In [26]:


# Lemos o ficheiro com as dimensões dos produtos
df_productDimensions = pd.read_csv('./dataset/product_hierarchy.csv')

# Verificamos o dataframe obtido
print(df_productDimensions.head())



  product_id  product_length  product_depth  product_width cluster_id  \
0      P0000             5.0           20.0           12.0        NaN   
1      P0001            13.5           22.0           20.0  cluster_5   
2      P0002            22.0           40.0           22.0  cluster_0   
3      P0004             2.0           13.0            4.0  cluster_3   
4      P0005            16.0           30.0           16.0  cluster_9   

  hierarchy1_id hierarchy2_id hierarchy3_id hierarchy4_id hierarchy5_id  
0           H00         H0004       H000401     H00040105   H0004010534  
1           H01         H0105       H010501     H01050100   H0105010006  
2           H03         H0315       H031508     H03150800   H0315080028  
3           H03         H0314       H031405     H03140500   H0314050003  
4           H03         H0312       H031211     H03121109   H0312110917  


In [27]:


# Adicionamos uma nova coluna com a média de desempenho das previsões de receitas e vendas de cada modelo aplicado a cada produto
df_productPredictions['average_r2'] = (df_productPredictions['r2_sales'] + df_productPredictions['r2_revenue']) / 2

# Agrupamos os dados por produto, escolhendo apenas o modelo com a melhor média de desempenho das previsões, com o intuito de escolhermos os valores mais precisos
df_productPredictions = df_productPredictions.groupby('product_id').apply(lambda x: x.loc[x['average_r2'].idxmax()]).reset_index(level=0, drop=True)

# Verificamos a distribuição dos modelos escolhidos
print(df_productPredictions.model.value_counts().head())



LinearRegression         592
RandomForestRegressor      4
XGBRegressor               3
Name: model, dtype: int64


In [28]:


# Juntamos as dimensões dos produtos
df = pd.merge(df_productPredictions, df_productDimensions, left_on='product_id', right_on='product_id', how='left')

# Adicionamos uma nova coluna com as dimensões cúbicas (i.e. volume)
df['dimensions'] = (df['product_length'] * df['product_depth'] * df['product_width'])

# Removemos as colunas desnecessárias
df = df[['product_id', 'periodicity_total_pred_sales', 'periodicity_total_pred_revenue', 'dimensions']]

# Verificamos o dataframe final
print(df.head())



  product_id  periodicity_total_pred_sales  periodicity_total_pred_revenue  \
0      P0001                            20                          196.86   
1      P0002                             1                          230.01   
2      P0004                             1                            0.85   
3      P0005                            12                          499.93   
4      P0006                             1                            0.00   

   dimensions  
0      5940.0  
1     19360.0  
2       104.0  
3      7680.0  
4      1912.5  


In [65]:
#-------------------------------User Input-------------------------------------

#add user input for the multiple variables

minAmountAllProducts = int(input("Por favor insira a quantidade mínima global de produtos: "))
maxAmountAllProducts = int(input("Por favor insira a quantidade máxima global de produtos: "))

minAmountPerProduct = int(input("Por favor insira a quantidade mínima de cada produto: "))
maxAmountPerProduct  = int(input("Por favor insira a quantidade máxima de cada produto: "))

minProductVariety = int(input("Por favor insira a variedade mínima de produtos: "))
maxProductVariety  = int(input("Por favor insira a variedade máxima de produtos: "))

storeDimensions  = int(input("Por favor insira as dimensões úteis da loja (i.e. volume): "))

In [81]:


# Calcula as receitas previstas de um produto tendo em consideração a quantidade escolhida e a quantidade prevista
def getPredictedProfit(productIndex, chosenQuantity):
    product = df.iloc[productIndex]
    if product.periodicity_total_pred_sales == 0:
        return 0
    return min(product.periodicity_total_pred_revenue, (product.periodicity_total_pred_revenue / product.periodicity_total_pred_sales) * chosenQuantity)

# Verificamos as receitas previstas para o primeiro produto, tendo em conta vários valores diferentes, sendo que deverá limitar as receitas consoante as unidades vendidas previstas
print(f"Unidades Vendidas Previstas do Produto {df.iloc[0].product_id}: {df.iloc[0].periodicity_total_pred_sales}")
print(f"Receitas Previstas para 5 unidades escolhidas: {getPredictedProfit(0, 5)}")
print(f"Receitas Previstas para 10 unidades escolhidas: {getPredictedProfit(0, 10)}")
print(f"Receitas Previstas para 15 unidades escolhidas: {getPredictedProfit(0, 15)}")
print(f"Receitas Previstas para 20 unidades escolhidas: {getPredictedProfit(0, 20)}")
print(f"Receitas Previstas para 25 unidades escolhidas: {getPredictedProfit(0, 25)}")




Unidades Vendidas Previstas do Produto P0001: 20
Receitas Previstas para 5 unidades escolhidas: 49.215
Receitas Previstas para 10 unidades escolhidas: 98.43
Receitas Previstas para 15 unidades escolhidas: 147.645
Receitas Previstas para 20 unidades escolhidas: 196.86
Receitas Previstas para 25 unidades escolhidas: 196.86


In [153]:


# Definimos as variáveis de decisão, onde serão armazenadas as quantidades escolhidas
decisionVariables = []
binaryVariables = []

# create a binary variable for each product
for product_id in df.product_id:
    decisionVariable = {product_id:  pulp.LpVariable(product_id, lowBound=0, cat='Integer')}
    decisionVariables.append(decisionVariable)
    binaryVariable = {product_id:  LpVariable(f'used_{product_id}', cat='Binary')}
    binaryVariables.append(binaryVariable)


print(decisionVariables)
print(binaryVariables)



[{'P0001': P0001}, {'P0002': P0002}, {'P0004': P0004}, {'P0005': P0005}, {'P0006': P0006}]
[{'P0001': used_P0001}, {'P0002': used_P0002}, {'P0004': used_P0004}, {'P0005': used_P0005}, {'P0006': used_P0006}]


In [154]:


# Função objetivo
def getObjectiveFunction():
    model = 0
    for index, row in df.iterrows():
        model += (getPredictedProfit(index, row.periodicity_total_pred_sales) * decisionVariables[index][row.product_id])
    return model

# Criação do modelo ILP
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

# Adicionamos a função objetivo ao modelo
model += getObjectiveFunction()

# Verificamos o modelo
print(model)



ILP_Problem:
MAXIMIZE
196.86*P0001 + 230.01*P0002 + 0.85*P0004 + 499.93*P0005 + 0.0
VARIABLES
0 <= P0001 Integer
0 <= P0002 Integer
0 <= P0004 Integer
0 <= P0005 Integer



In [155]:
df = df.head(5)

In [157]:


# Função que aplica todas as restrições pretendidas para o modelo de otimização
def applyRestrictionsToModel(model):
    # Aplicamos a restrição de quantidades globais mínimas e máximas dos produtos
    model += pulp.lpSum([decisionVariables[index][row.product_id] for index, row in df.iterrows()]) <= maxAmountAllProducts
    model += pulp.lpSum([decisionVariables[index][row.product_id] for index, row in df.iterrows()]) >= minAmountAllProducts

    # Aplicamos a restrição de dimensões globais máximas dos produtos, sendo que não deverão exceder as dimensões da loja
    #model += pulp.lpSum([row.dimensions*decisionVariables[index][row.product_id] for index, row in df.iterrows()]) <= storeDimensions

    # Aplicamos a restrição de variedade globais mínimas e máximas dos produtos
    # model += pulp.lpSum([binaryVariables[index][row.product_id] for index, row in df.iterrows()]) <= maxProductVariety
    # model += pulp.lpSum([binaryVariables[index][row.product_id] for index, row in df.iterrows()]) >= minProductVariety

    # Aplicamos a restrição de quantidades individuais mínimas e máximas dos produtos
    for index, row in df.iterrows():
        model += decisionVariables[index][row.product_id] <= maxAmountPerProduct
        model += decisionVariables[index][row.product_id] >= minAmountPerProduct

# Verificamos as restrições
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)
model += getObjectiveFunction()
applyRestrictionsToModel(model)
print(model)



ILP_Problem:
MAXIMIZE
196.86*P0001 + 230.01*P0002 + 0.85*P0004 + 499.93*P0005 + 0.0
SUBJECT TO
_C1: P0001 + P0002 + P0004 + P0005 + P0006 <= 10000

_C2: P0001 + P0002 + P0004 + P0005 + P0006 >= 10

_C3: P0001 <= 1000

_C4: P0001 >= 10

_C5: P0002 <= 1000

_C6: P0002 >= 10

_C7: P0004 <= 1000

_C8: P0004 >= 10

_C9: P0005 <= 1000

_C10: P0005 >= 10

_C11: P0006 <= 1000

_C12: P0006 >= 10

VARIABLES
0 <= P0001 Integer
0 <= P0002 Integer
0 <= P0004 Integer
0 <= P0005 Integer
0 <= P0006 Integer



In [158]:
minAmountAllProducts = 10
maxAmountAllProducts = 10000
minAmountPerProduct = 10
maxAmountPerProduct  = 1000
minProductVariety = 100
maxProductVariety  = 1000
storeDimensions = 1000000

# Executamos o modelo de otimização
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)
model += getObjectiveFunction()
applyRestrictionsToModel(model)
status = model.solve()

# Verificamos o sucesso do modelo, isto é, se conseguiu obter uma solução ótima
print(f"Resultado do Modelo: {pulp.LpStatus[status]}")

# Verificamos a solução obtida pelo modelo, isto é, as receitas previstas para as quantidades escolhidas
print(f"\nTotal de Receitas Previstas ($): \n{round(pulp.value(model.objective) / 1000000, 2)}M")


Resultado do Modelo: Optimal

Total de Receitas Previstas ($): 
0.93M


In [159]:

# Verificamos as quantidades escolhidas pelo modelo para cada produto
for index, row in df.iterrows():
    print(f"Quantidade ótima do produto {row.product_id}: {pulp.value(decisionVariables[index][row.product_id])}")



Quantidade ótima do produto P0001: 1000.0
Quantidade ótima do produto P0002: 1000.0
Quantidade ótima do produto P0004: 1000.0
Quantidade ótima do produto P0005: 1000.0
Quantidade ótima do produto P0006: 10.0


----------- Genetic Algorithm -------------

    Restritions:
    

Restrictions functions return 1 respected, 0 if not respected


In [8]:

#Total Amount Restriction
def validateTotalAmount(individuo):
    somaTotalAmountIndividuo = sum(individuo)

    if somaTotalAmountIndividuo < minTotalAmount
        return somaTotalAmountIndividuo - minTotalAmount

    if somaTotalAmountIndividuo > maxTotalAmount
        return somaTotalAmountIndividuo - maxTotalAmount

    return 0

In [9]:
#Total Quantity Restriction
def validateTotalQuantity(individuo):
    contador = 0
    individuoLen = len(individuo)
    for indAux in individuo:
        if (indAux >= minAmountProduct) and (indAux <= maxAmountProduct):
            contador = contador + 1
    if(contador == individuoLen):
        return 1
    else:
        return 0

In [10]:
#Dimension Restriction
def validateDimension(individuo):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * products[contador]['dimension']
        contador = contador + 1
    if(totalDimension > storeDimensions):
        return 0
    else:
        return 1

Creation of First Population

In [21]:
print(createFirstPopulation([],products,2))

[[3, 3, 1, 3], [3, 2, 2, 3]]


Evaluation Function

In [22]:
#get profict of product with quantity 
def getPredictedProfit(product, chosenQuantity):
    return min(product['predictedProfits'], (product['predictedProfits']/product['predictedProductQuantities']) * chosenQuantity)

In [74]:

#Evaluates individuos by their profict

#parameters entry:
    #list population with values: [[1,2,3,4],[1,7,5,4]]
    #dictionary products [{'productName': 'p1', 'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5},...]
    #listPopulationWithProfit []
    
# returns dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
def createFitnessFunction(population, products,listPopulationWithIndiviuoAndProfit):

    #population= [[1,2,3,4],[1,7,5,4]]
    for individuo in population:

        individuoProfit = 0
        contador = 0
        #individuo= [1,2,3,4]
        for productQuantity in individuo:

            product = products[contador]
            individuoProfit = individuoProfit + getPredictedProfit(product, productQuantity)
            listPopulationWithProfitAux = {'individuo': individuo, 'calculatedProfit':individuoProfit}
            contador = contador + 1

            
        listPopulationWithIndiviuoAndProfit.append(listPopulationWithProfitAux)


    return listPopulationWithIndiviuoAndProfit

In [75]:
listPopulationWithProfit=createFirstPopulation([],products,10)
listPopulationWithIndiviuoAndProfit=createFitnessFunction(listPopulationWithProfit, products,[])

print(listPopulationWithIndiviuoAndProfit)

[{'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}, {'individuo': [2, 3, 3, 2], 'calculatedProfit': 90.0}, {'individuo': [3, 1, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [2, 2, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [2, 3, 3, 3], 'calculatedProfit': 90.0}]


Crossover

In [114]:
#Evaluates individuos by their profict

#parameters entry:
    #listPopulationWithIndiviuoAndProfit with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns:
    # listPopulationWithProfitOrdered same as listPopulationWithIndiviuoAndProfit but Ordered  
    # populationOrderedByProfit population ordered [[3, 2, 2, 3],..]
def getListsOrderByProfit(listPopulationWithIndiviuoAndProfit):
    listPopulationWithProfitOrdered=sorted(listPopulationWithIndiviuoAndProfit,key=lambda x: x['calculatedProfit'], reverse=True)
    populationOrderedByProfit=[d['individuo'] for d in listPopulationWithProfitOrdered]
    # print(listPopulationWithProfitOrdered)
    # print(populationOrderedByProfit)
    return listPopulationWithProfitOrdered,populationOrderedByProfit

In [115]:
listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
print(listPopulationWithProfitOrdered)
print(populationOrderedByProfit)

[{'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [2, 3, 3, 2], 'calculatedProfit': 90.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [2, 3, 3, 3], 'calculatedProfit': 90.0}, {'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 1, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [2, 2, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}]
[[3, 3, 3, 3], [3, 3, 3, 3], [2, 3, 3, 2], [3, 3, 2, 3], [3, 3, 2, 3], [2, 3, 3, 3], [3, 2, 2, 3], [3, 1, 3, 3], [2, 2, 3, 3], [3, 2, 2, 3]]


In [122]:

#parameters entry:
    #list initialPopulation with values: [[1,2,3,4],[1,7,5,4]] already order by profi
    #list listPopulationGoingToCrossover: []
    #crossoverTax: tax to calculate numer of elements to do crossover ex: 0.10-> 90% do crossover
    #listPopulationWithIndiviuoAndProfit: dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns finalPopulation- list that does not  : [[1,2,3,4],[1,7,5,4]] that will then go to mutation
def getCrossoverListAccordingToCrossoverTax(initialPopulation,crossoverTax):

    numberElementsNoCrossover = round(len(initialPopulation)*crossoverTax)

    #EX:[[1,2,3,4],[1,7,5,4]]
    finalPopulation = [individuo for individuo in initialPopulation[:numberElementsNoCrossover]]
    #EX:[[1,2,3,4],[1,7,5,4]
    listCrossOver = [individuo for individuo in initialPopulation[numberElementsNoCrossover+1:]]

    return finalPopulation,listCrossOver

In [121]:
listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
finalPopulation,listCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,0.10)
print(finalPopulation)
print(listCrossOver)




[[3, 3, 3, 3]]
[[2, 3, 3, 2], [3, 3, 2, 3], [3, 3, 2, 3], [2, 3, 3, 3], [3, 2, 2, 3], [3, 1, 3, 3], [2, 2, 3, 3], [3, 2, 2, 3]]


In [126]:
#parameters entry:
    #listToCrossOver: [[1,2,3,4],[1,7,5,4]] already order by profi

# returns listpopulationAfterCrossover [[1,2,3,4],[1,7,5,4]] already changed after crossover
def crossoverFunction(listToCrossOver):

    listpopulationAfterCrossover=[]

    while len(listToCrossOver)>0:
    
        if len(listToCrossOver) == 1:
            #listToCrossOver[0]= last individuo remaining [1,2,3,4]
            listpopulationAfterCrossover.append(listToCrossOver[0])
            listToCrossOver.remove(listToCrossOver[0])
            
        else:
            
            #generates random numbers to discover wich individuos ae going to crossover
            indexFirstPai = random.randint(0, len(listToCrossOver)-1)
            indexSecondPai = random.randint(0, len(listToCrossOver)-1)

            #garantees that is not selected the same individu
            while indexFirstPai == indexSecondPai:
                indexSecondPai = random.randint(0, len(listToCrossOver)-1)
            
            #individuo to crossover
            firstPai = listToCrossOver[indexFirstPai]
            secondPai = listToCrossOver[indexSecondPai]
            
            #individuo saved to after remove from list
            firstPaiToDelete = listToCrossOver[indexFirstPai]
            secondPaiToDelete = listToCrossOver[indexSecondPai]
            
            crossoverPoint = random.randint(1, len(listToCrossOver[indexFirstPai])-1)
            
            for i in list(range(crossoverPoint)):
                aux = firstPai[i]
                firstPai[i] = secondPai[i]
                secondPai[i] = aux

            diff_quant = validateInd(firstPai) 
            while diff_quant != 0:
                adjustQuantities(firstPai, diff_quant)
                diff_quant = isIndValid(firstPai)

            while diff_quant != 0:
                adjustQuantities(secondPai, diff_quant)
                diff_quant = isIndValid(secondPai)

            #validate that first child is good
            # totalAmountRestriction1 = validateTotalAmount(firstPai)
            # totalQuantityRestriction1 = validateTotalQuantity(firstPai)
            # dimensionRestriction1 = validateDimension(firstPai, products)
            # #validate that second child is good
            # totalAmountRestriction2 = validateTotalAmount(secondPai)
            # totalQuantityRestriction2 = validateTotalQuantity(secondPai)
            # dimensionRestriction2 = validateDimension(secondPai, products)

            # if (totalAmountRestriction1 == 1) and (totalQuantityRestriction1 == 1) and (dimensionRestriction1 == 1) and (totalAmountRestriction2 == 1) and (totalQuantityRestriction2 == 1) and (dimensionRestriction2 == 1):
                
            listpopulationAfterCrossover.append(firstPai)
            listpopulationAfterCrossover.append(secondPai)
            listToCrossOver.pop(firstPaiToDelete)
            listToCrossOver.remove(secondPaiToDelete)

            

    return listpopulationAfterCrossover


# Returna 0 se for válido
# Return um valor diferente de 0 (+ ou -) consoante a maior diferença absoluta de quantidades globais inválida encontrada 
def validateInd(ind):
    #validate that second child is good
    diffGlobalQuantities = validateTotalAmount(ind)

    # totalQuantityRestriction = validateTotalQuantity(ind)
    dimensionRestriction = validateDimension(ind, products)
    if (diffGlobalQuantities != 0) and (dimensionRestriction == 1):
        return True
    return False

def adjustQuantities(ind, diff_quant):
    indWithPredictions = pd.DataFrame(products, columns=['predictedProductQuantities', 'predictedProfits'])
    indWithPredictions['chosen_quantity'] = ind
    # Armazena a diferença entre a quantidade prevista de vendas e a quantidade escolhida
    # Exemplo: 
    # Quantidade Prevista = 10
    # Quantidade Escolhida = 5
    # Diferença = 10 - 5 = 5
    indWithPredictions['quantity_diff'] = indWithPredictions['predictedProductQuantities'] - indWithPredictions['chosen_quantity']
    
    # Se existirem quantidades globais em excesso então iremos remover
    if diff_quant > 0:
        # Ordenamos os produtos por ordem ascendente dos que têm mais quantidades em excesso (> Quantidades Previstas)
        # e, de seguida, pelos que têm menos receitas previstas dado que iremos remover quantidades
        indWithPredictions = indWithPredictions.sort_values(by=['quantity_diff', 'predictedProfits'], ascending=[True, True])
        indexToAdjust = 0
        while diff_quant > 0:
            quantityToAdjust = indWithPredictions.iloc[indexToAdjust]['quantity_diff']
            if quantityToAdjust > diff_quant:
                quantityToAdjust = diff_quant
            ind[indexToAdjust]
            ind[indexToAdjust] -= quantityToAdjust
            diff_quant -= quantityToAdjust
            indexToAdjust += 1
    # Se existirem quantidades globais em falta então iremos adicionar
    if diff_quant < 0:
        # Ordenamos os produtos por ordem descendente dos que têm mais quantidades em falta (< Quantidades Previstas) 
        # e, de seguida, pelos que têm mais receitas previstas dado que iremos adicionar quantidades
        indWithPredictions = indWithPredictions.sort_values(by=['quantity_diff', 'predictedProfits'], ascending=[False, False])
        indexToAdjust = 0
        while diff_quant < 0:
            quantityToAdjust = abs(indWithPredictions.iloc[indexToAdjust]['quantity_diff'])
            if quantityToAdjust > abs(diff_quant):
                quantityToAdjust = abs(diff_quant)
            ind[indexToAdjust] += quantityToAdjust
            diff_quant += quantityToAdjust
            indexToAdjust += 1
    
    return ind



In [14]:
print(list(range(1)))

[0]


In [127]:
listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
finalPopulation,listCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,0.10)

print(crossoverFunction(listCrossOver))

8
6
4
4
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


KeyboardInterrupt: 

In [ ]:
#parameters entry:
    #list population with values: [[1,2,3,4],[1,7,5,4]]
    #crossoverTax: tax to calculate numer of elements to do crossover ex: 0.10-> 90% do crossover
    #listPopulationWithIndiviuoAndProfit: dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] that will then go to mutation and population ordered [[1,2,3,4],[1,7,5,4]]
def crossover(listPopulationWithIndiviuoAndProfit,crossoverTax):

    #function that returns dic ordered [{'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, and 
    listPopulationWithProfitOrdered,populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)

    #list with population (size according to crossoverTax) after crossover
    listCrossover=[]
    #function that returns final list of population that doesn't go to crossover[[1,2,3,4],[1,7,5,4]] and list that goes to crossover[[1,2,3,4],[1,7,5,4]]
    finalPopulation,listToCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,crossoverTax)

    #crossover function that receives list listToCrossOver and returns list after crossover [[1,2,3,4],[1,7,5,4]]
    listpopulationAfterCrossover=crossoverFunction(listToCrossOver)
    
    #population with individuos after crossover being added to the finalpopulation list that is going to go to mutation (size=initialSize)
    finalPopulation.append(listpopulationAfterCrossover)

    return finalPopulation


Mutation

In [ ]:
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    
# returns listPopulationToMutate : [[1,2,3,4],[1,7,5,4]] after mutation
def getMutationListAccordingToMutationTax(finalPopulation,taxMutation):
    mutationList=finalPopulation[:]
    for x in mutationList:
        #generates random number 
        t=random.randint(0, 100)

        #when random number if > taxMutation then removes values from list, because this is has high probability the list of to mutate is going to be small has expected
        if t >= taxMutation:
            mutationList.remove(x)
    return mutationList

In [ ]:
def mutationFunction(finalPopulation,listPopulationToMutate,taxChangeGene):
    for el in finalPopulation:
        
        initialEl=el
        
        for l in listPopulationToMutate:
            
            if el==l:
                index=random.randint(0, len(listPopulationToMutate[0])-1)
                
                randomSumOrLess = random.randint(0, 1)
                
                if(randomSumOrLess == 1):
                    el[index] = el[index] + round(el[index]*taxChangeGene)
                else:
                    el[index] = el[index] - round(el[index]*taxChangeGene)
                
                        
                totalAmountRestriction = validateTotalAmount(el)
                totalQuantityRestriction = validateTotalQuantity(el)
                dimensionRestriction = validateDimension(el, products)
                        
                if (totalAmountRestriction == 0) or (totalQuantityRestriction == 0) or (dimensionRestriction == 0):
                    el=initialEl
                    
    return finalPopulation

In [ ]:
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    #changeGene: tax to calculate how the quantity of a product is going to change, ex: 0.1 QP1=2 QP1=2*2*0.1
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
def mutation(finalPopulation, taxMutation, taxChangeGene):

    #function that returns final list of population that doesn't go to crossover[[1,2,3,4],[1,7,5,4]] and list that goes to crossover[[1,2,3,4],[1,7,5,4]]
    listPopulationToMutate=getMutationListAccordingToMutationTax(finalPopulation,taxMutation)

    #mutation function that receives list listPopulationToMutate and returns list after mutation [[1,2,3,4],[1,7,5,4]]=Final population to go then to next generation
    listpopulationAfterMutation=mutationFunction(finalPopulation,listPopulationToMutate,taxChangeGene)
    
    return listpopulationAfterMutation

In [21]:
#Create first population 


#parameters entry:
    #list population []
    #dictionary products [{'productName': 'p1', 'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5},...]
    #populationSize ex:2
#return: list of products with theirs quantities [[3, 3, 3, 1], [3, 1, 3, 3]]
def createFirstPopulation(populationSize):
    pop = []

    while len(pop) < populationSize:
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        #generates the chromossomes each representing one product
        for j in range(len(products)):
            quantity = random.randint(0, round((maxAmountProduct/3)))
            solution.append(quantity)
            
        #Total Amount Restriction
        totalAmountRestriction = validateTotalAmount(solution)
        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution)
        
        if (totalAmountRestriction == 1) and (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            pop.append(solution)
        
    return pop

print(createFirstPopulation(10).head())

NameError: name 'products' is not defined

Iterate multiple times

In [ ]:
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    #changeGene: tax to calculate how the quantity of a product is going to change, ex: 0.1 QP1=2 QP1=2*2*0.1
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
def generatePopulation(previousPop, populationSize, crossoverTax, taxMutation, taxChangeGene):
    #creates dic with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    listPopulationWithIndiviuoAndProfit=createFitnessFunction(listPopulationWithProfit, previousPop,[])
    
    #returns listpopulationAfterCrossover [[1,2,3,4],[1,7,5,4]] already changed after crossover
    listAfterCrossover=crossover(listPopulationWithIndiviuoAndProfit,crossoverTax)

    #returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
    listAfterMutation=mutation(listAfterCrossover, taxMutation, taxChangeGene)

    return listAfterMutation

    

In [ ]:
def runGeneticAlgorithm(popSize, nIterations, crossoverTax, taxMutation, taxChangeGene):
    it = 0
    popsList = []
    
    while it < nIterations:
        if it == 0:
            #creates first population with random quantities
            firstPop=createFirstPopulation(popSize)
            popsList.append(firstPop)
        else:
            newPop = generatePopulation(popSize, crossoverTax, taxMutation, taxChangeGene)
            popsList.append(newPop)
        it += 1


nIterations = random.randint(0, 100)
print(nIterations)

popSize = random.randint(1,100)
print(popSize)

crossoverTax = round(random.uniform(0.01, 0.10),2)
print(crossoverTax)

taxMutation = random.randint(1,5)
print(taxMutation)

taxChangeGene = round(random.uniform(0.01,0.1),2)
print(taxChangeGene)

runGeneticAlgorithm(popSize, nIterations, crossoverTax, taxMutation, taxChangeGene)




In [19]:
# def time_plot(valuesListToCompare, label):
#     fig, ax = plt.subplots(figsize=(15,5))
#     sns.lineplot(x = values1.index.values, y = values1, data=data, ax=ax, color='blue', label=label)
#     sns.lineplot(x = values2.index.values, y = values2, data=data, ax=ax, color='red', label=label)
#     sns.lineplot(x = values3.index.values, y = values3, data=data, ax=ax, color='orange', label=label)
#     sns.lineplot(x = values4.index.values, y = values3, data=data, ax=ax, color='pink', label=label)



# values1 =  geneticAlgorithm(products, populationSize,crossoverTax,taxMutation, 0.7)
# values2 =  geneticAlgorithm(products, populationSize,crossoverTax,taxMutation, 0.8)
# values3 =  geneticAlgorithm(products, populationSize,crossoverTax,taxMutation, 0.5)
# values4 =  geneticAlgorithm(products, populationSize,crossoverTax,taxMutation, 0.6)

# time_plot(values1)

NameError: name 'geneticAlgorithm' is not defined

In [17]:
 = 

SyntaxError: invalid syntax (2304172042.py, line 1)